<a href="https://colab.research.google.com/github/Devhsc/HSC-Project/blob/main/DATA3406_A2_Product_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test

Importing packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Load Data

In [ ]:
# Load the data
daily_steps_df = pd.read_csv('../Data/dailySteps_merged.csv')
hourly_steps_df = pd.read_csv('../Data/hourlySteps_merged.csv')
minute_steps_df = pd.read_csv('../Data/minuteStepsWide_merged.csv')

Cleaning data for minute file

In [ ]:
# Select the columns that represent steps per minute from step00 to step59
step_columns = [f"Steps{i:02d}" for i in range(60)]

# Flatten the DataFrame to a single series to find the average of each step
all_steps = minute_steps_df[step_columns].values.flatten()
non_zero_steps = all_steps[all_steps > 0]

# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = pd.Series(non_zero_steps).quantile(0.25)
Q3 = pd.Series(non_zero_steps).quantile(0.75)

# Compute the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
filtered_steps = non_zero_steps[(non_zero_steps >= lower_bound) & (non_zero_steps <= upper_bound)]

# selected individual id
selected_id = 2022484408

# Filter the DataFrame to get the data for the selected individual
filtered_minute_steps_df = minute_steps_df[minute_steps_df['Id'] == selected_id]

filtered_minute_steps_df['ActivityHour'] = pd.to_datetime(filtered_minute_steps_df['ActivityHour'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
filtered_minute_steps_df.loc[:, 'ActivityHour'] = pd.to_datetime(filtered_minute_steps_df['ActivityHour'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

# Create separate dataframes for 7-9 AM and 5-7 PM
am_commuting_hours_df = filtered_minute_steps_df.loc[
    (filtered_minute_steps_df['ActivityHour'].dt.hour >= 7) & (filtered_minute_steps_df['ActivityHour'].dt.hour <= 9)
    ].copy()

pm_commuting_hours_df = filtered_minute_steps_df.loc[
    (filtered_minute_steps_df['ActivityHour'].dt.hour >= 17) & (filtered_minute_steps_df['ActivityHour'].dt.hour <= 19)
    ].copy()